In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil import parser

import mysql.connector

# Functions
from uploadDB import insert_channel_in_DB, remove_emojis, remove_links_and_line_jumps, remove_accents, clean_video_statics, video_tables_db_upload

import warnings
warnings.filterwarnings("ignore")

# Getting the files in the folder:

In [2]:
sheets = ['channel_evolution', 'video_statics', 'video_variables']

In [3]:
# The list of items
files = os.listdir("./extractions")
 
# Loop to print each filename separately
for filename in files:
    print(filename)

Extraction_2023-05-02.xlsx
Extraction_2023-05-03.xlsx
Extraction_2023-05-05.xlsx
Extraction_2023-05-08.xlsx
Extraction_2023-05-09.xlsx
Extraction_2023-05-10.xlsx
Extraction_2023-05-11.xlsx
Extraction_2023-05-12.xlsx
Extraction_2023-05-15.xlsx
Extraction_2023-05-16.xlsx
Extraction_2023-05-17.xlsx
Extraction_2023-05-18.xlsx
Extraction_2023-05-19.xlsx
Extraction_2023-05-23.xlsx
Extraction_2023-05-24.xlsx
Extraction_2023-05-25.xlsx
Extraction_2023-05-26.xlsx


In [4]:
# Reading each file in dict
dump = {}

for i,filename in enumerate(files):

    print(filename)

    for sheet in sheets:

        print('--->', sheet)

        # First Read
        if i == 0:
            dump[sheet] = pd.read_excel('extractions/' + filename, sheet_name=sheet, index_col=[0])
        else:
            helpDF = pd.read_excel('extractions/' + filename, sheet_name=sheet,index_col=[0])

            dump[sheet] = pd.concat([dump[sheet], helpDF], ignore_index=False)

Extraction_2023-05-02.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-03.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-05.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-08.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-09.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-10.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-11.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-12.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-15.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-16.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-17.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction

In [5]:
insert_channel_in_DB(dump, chop_details = True)

Inserted: channel_evolution


Video Level

In [6]:
dump2 = {
    'video_statics' : dump['video_statics'].reset_index().copy(),
    'video_variables' : dump['video_variables'].reset_index().copy()
}

# Keeping only one version of the extractions
dump2['video_statics'].drop_duplicates(subset=['ChannelID','video_id','CreatedOn'], keep='first', inplace=True)
dump2['video_statics'].drop_duplicates(subset=['ChannelID','video_id'], keep='first', inplace=True)

In [7]:
# Get all the video_statics already in the DB to clean

# Connect to the database
cnx = mysql.connector.connect(host='localhost',
                                        database='youtubeproject',
                                        user='root',
                                        password='optim123*')

sql_query = """SELECT video_id from video_statics"""
frstatics = pd.read_sql(sql_query, cnx)
vids_in_statics = frstatics['video_id'].to_list()

cnx.close()

# Filtering
cleaning_mask = ~dump2['video_statics']['video_id'].isin( vids_in_statics )

dump2['video_statics'] = dump2['video_statics'][cleaning_mask]
#dump2['video_statics']

In [8]:
# cleaning here instead on function
dump2['video_statics']['title'] = dump2['video_statics']['title'].apply(remove_emojis).apply(remove_links_and_line_jumps).apply(remove_accents)
dump2['video_statics']['descr'] = dump2['video_statics']['descr'].astype(str).apply(remove_emojis).apply(remove_links_and_line_jumps).apply(remove_accents)

#dump2['video_statics']

In [9]:
video_tables_db_upload( dump2 )

Inserting: video_statics...
Inserted: video_statics
Inserting: video_variables...
Inserted: video_variables
